# Сессия 4 

импорты для подготовки 

In [1]:
import pandas as pd
import pickle as pkl
from sklearn.ensemble import RandomForestClassifier

In [7]:
train = pd.read_csv("train.csv", index_col=0)
test =  pd.read_csv("test.csv", index_col=0)
test_ses = pd.read_csv("КЗ Юниоры/Сессия1/test_ses.csv")


Создание файла как пример использования в будущей программе

In [12]:
example = test_ses.iloc[:1000]

In [13]:
example.to_csv("example.csv")

In [5]:
X = train.drop(["session_id", "target"], axis=1)
y = train.target

In [6]:
RFC = RandomForestClassifier(random_state=42,n_estimators=500, max_features="auto", max_depth=15, criterion="gini", bootstrap=False)

Обучение модели и сохранение ее как файл pkl

In [8]:
RFC.fit(X, y)

RandomForestClassifier(bootstrap=False, max_depth=15, n_estimators=500,
                       random_state=42)

In [ ]:
pkl_filename = "RFC.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(RFC, file)

Подготовка Функций к созданию программы
1. load_model(filename) - загрузка модели
2. new_features(df) - добавления новых признаков
3. preproc(df) - предобработка файла с данными
4. make_pred(model, df) - функция предикта таргетов 
5. helpp() - функция вызова справки

In [36]:
def load_model(filename):
    print(">>> Загрузка модели")
    with open(filename, 'rb') as file:
            model = pickle.load(file)
    print(">>> Готово")
    return model

def new_features(df):
    df["ym"] = df.apply(lambda x: "".join(x["time1"][:x["time1"].rfind("-")].split("-")), axis=1)
    
    df["time_diff"] = df.apply(lambda x: (datetime.datetime.strptime(x["time10"], '%Y-%m-%d %H:%M:%S') -
 datetime.datetime.strptime(x["time1"], '%Y-%m-%d %H:%M:%S')).total_seconds(), axis=1)
    
    df["max_repeat"] = df[["site1","site2","site3","site4","site5","site6","site7","site8","site9","site10"]].apply(lambda x: x.value_counts().max(), axis=1)
    
    df = df.astype({"ym":"int64"})
    return df 

def preproc(df):
    df = df[df['site10'].notna()]
    df = df.reset_index(drop=True)
    df = new_features(df)
    df.drop(["time1","time2","time3","time4","time5","time6","time7","time8","time9","time10"], axis=1, inplace=True)
    return df

def make_pred(model, df):
    pred = model.predict_proba(df.drop(["session_id"], axis=1))
    tar = []
    for i in pred:
        tar.append(i[1])

    Id = df.session_id

    sample = pd.DataFrame({"session_id":Id, "probability":tar, "target":[int(round(i)) for i in tar]})
    return sample

def helpp():
    print("""Несколько правил для успешной работы программы: \n 
            1. Файл с вашими данными сессии должен находиться в той же папке что и программа 
            2. Все файлы которые вы указываете необходимо указывать вместе с раширением 
            3. Вы можете набрать exit для выхода из программы 
            4. Вместе с программой идет файл RFC.pkl - это уже обученая модель, если вы хотите заменить на другую просто замените ее в папке 
            5. Так же есть файл example.csv с примером данных""")

Для запуска программы вы можете запустить ее из ячейки ниже, либо открыть командную строку, перейти в нужную директорию(cd), и вызвать программу командой `python programm.py`

In [37]:
print("Loading modules...")
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
import pickle
import time
import numpy as np
import datetime
from goto import with_goto

print("Done")
print()

@with_goto
def main():
    while True:
        print("Добро пожаловать в приложение. Для правильной работы следуйте указаниям. Введите help для получения справки.")
        label .restart
        a = input(">>> Введите название вашего файла(с расширением): ")
        if a == "help":
            helpp()
            goto .restart
        elif a == "exit":
            break
        elif "csv" in a:
            print(">>> Ваш файл ",a,"?(y/n)")
            b=input()
            if b == "help":
                helpp()
                goto .restart
            elif b == "exit":
                break
            elif b == "y":
                try:
                    df = pd.read_csv(a, index_col=0)
                except e:
                    print("что то не так, проверьте имя файла еще раз")
                    goto .restart
            else: 
                goto .restart
        else:
            print(">>> error0")
            goto .restart
        
        print(">>> Обработка данных")
        df = preproc(df)
        label .loadmodel
        model = load_model("RFC.pkl")
        R = make_pred(model, df)
        print("<========RESULT========>")
        
        print(R.head())
        print("5 out of", R.shape[0])
        print("<========RESULT========>")
        
        label .save
        a = input(">>> Хотите сохранить все в файл?(y/n): ")
        if a == "help":
            helpp()
            goto .save
        elif a == "exit":
            break
        elif a=="n":
            break
        elif a=="y":
            fn = input(">>> Введите имя csv файла c расширением: ")
            if fn == "help":
                helpp()
                goto .save
            elif fn == "exit":
                break
            elif "csv" in fn:
                R.to_csv(fn)
            else: 
                print(">>> что то не так")
                goto .save
            print(">>> Готово")
        break
main()

Loading modules...
Done

Добро пожаловать в приложение. Для правильной работы следуйте указаниям. Введите help для получения справки.
>>> Введите название вашего файла(с расширением): help
Несколько правил для успешной работы программы: 
 
            1. Файл с вашими данными сессии должен находиться в той же папке что и программа 
            2. Все файлы которые вы указываете необходимо указывать вместе с раширением 
            3. Вы можете набрать exit для выхода из программы 
            4. Вместе с программой идет файл RFC.pkl - это уже обученая модель, если вы хотите заменить на другую просто замените ее в папке 
            5. Так же есть файл example.csv с примером данных
>>> Введите название вашего файла(с расширением): example.csv
>>> Ваш файл  example.csv ?(y/n)
y
>>> Обработка данных
>>> Загрузка модели
>>> Готово
<========RESULT========>
   session_id  probability  target
0           1     0.030722       0
1           2     0.000000       0
2           3     0.000636     